In [1]:
import sys
import math
import tensorflow as tf
import keras
import keras.backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Conv2D, Dropout, AlphaDropout, Activation, BatchNormalization, Flatten, \
                                    Concatenate, PReLU, TimeDistributed, LSTM, Masking
from keras.callbacks import Callback, ModelCheckpoint, CSVLogger
#from keras_tqdm import TQDMNotebookCallback

sys.path.insert(0, "../../python")
from common import *

Using TensorFlow backend.


In [2]:
#K.set_floatx('float16')
#K.floatx()

In [3]:
class MaskedDense(Dense):
    def __init__(self, units, **kwargs):
        super(MaskedDense, self).__init__(units, **kwargs)
        
    def call(self, inputs, mask=None):
        base_out = super(MaskedDense, self).call(inputs)
        if mask is None:
            return base_out
        zeros = tf.zeros_like(base_out)
        return tf.where(mask, base_out, zeros)

In [4]:
class NetSetup:
    def __init__(self, activation, activation_shared_axes, dropout_rate, first_layer_size, last_layer_size, decay_factor,
                 kernel_regularizer, time_distributed):
        self.activation = activation
        self.activation_shared_axes = activation_shared_axes
        if activation == 'relu' or activation == 'PReLU' or activation == 'tanh':
            self.DropoutType = Dropout
            self.kernel_init = 'he_uniform'
            self.apply_batch_norm = True
        elif activation == 'selu':
            self.DropoutType = AlphaDropout
            self.kernel_init = 'lecun_normal'
            self.apply_batch_norm = False
        else:
            raise RuntimeError('Activation "{}" not supported.'.format(activation))
        self.dropout_rate = dropout_rate
        self.first_layer_size = first_layer_size
        self.last_layer_size = last_layer_size
        self.decay_factor = decay_factor
        self.kernel_regularizer = kernel_regularizer
        self.time_distributed = time_distributed
    
    def RecalcLayerSizes(self, n_input_features, width_factor, compression_factor, consider_dropout = True):
        drop_factor = 1 + self.dropout_rate if consider_dropout else 1
        self.first_layer_size = int(math.ceil(n_input_features * drop_factor * width_factor))
        self.last_layer_size = int(math.ceil(n_input_features * drop_factor * compression_factor))

In [5]:
def add_block_ending(net_setup, name_format, layer):
    if net_setup.apply_batch_norm:
        #floatx = K.floatx()
        #K.set_floatx('float32')
        norm_layer = BatchNormalization(name=name_format.format('norm'))
        if net_setup.time_distributed:
            norm_layer = TimeDistributed(norm_layer, name=name_format.format('norm'))
        norm_layer = norm_layer(layer)
        #K.set_floatx(floatx)
    else:
        norm_layer = layer
    if net_setup.activation == 'PReLU':
        activation_layer = PReLU(shared_axes=net_setup.activation_shared_axes,
                                 name=name_format.format('activation'))(norm_layer)
    else:
        activation_layer = Activation(net_setup.activation, name=name_format.format('activation'))(norm_layer)
    if net_setup.dropout_rate > 0:
        return net_setup.DropoutType(net_setup.dropout_rate, name=name_format.format('dropout'))(activation_layer)
    return activation_layer


In [6]:
def dense_block(prev_layer, kernel_size, net_setup, block_name, n):
    DenseType = MaskedDense if net_setup.time_distributed else Dense
    dense = DenseType(kernel_size, name="{}_dense_{}".format(block_name, n),
                      kernel_initializer=net_setup.kernel_init,
                      kernel_regularizer=net_setup.kernel_regularizer)
    if net_setup.time_distributed:
        dense = TimeDistributed(dense, name="{}_dense_{}".format(block_name, n))
    dense = dense(prev_layer)
    return add_block_ending(net_setup, '{}_{{}}_{}'.format(block_name, n), dense)

In [7]:
def reduce_n_features_1d(input_layer, net_setup, block_name):
    prev_layer = input_layer
    current_size = net_setup.first_layer_size
    n = 1
    while True:
        prev_layer = dense_block(prev_layer, current_size, net_setup, block_name, n)
        if current_size == net_setup.last_layer_size: break
        current_size = max(net_setup.last_layer_size, int(current_size / net_setup.decay_factor))
        n += 1
    return prev_layer

In [8]:
def dense_block_sequence(input_layer, net_setup, n_layers, block_name):
    prev_layer = input_layer
    current_size = net_setup.first_layer_size
    for n in range(n_layers):
        prev_layer = dense_block(prev_layer, current_size, net_setup, block_name, n+1)
    return prev_layer

In [9]:
def conv_block(prev_layer, filters, kernel_size, net_setup, block_name, n):
    conv = Conv2D(filters, kernel_size, name="{}_conv_{}".format(block_name, n),
                  kernel_initializer=net_setup.kernel_init)(prev_layer)
    return add_block_ending(net_setup, '{}_{{}}_{}'.format(block_name, n), conv)


In [10]:
def reduce_n_features_2d(input_layer, net_setup, block_name):
    conv_kernel=(1, 1)
    prev_layer = input_layer
    current_size = net_setup.first_layer_size
    n = 1
    while True:
        prev_layer = conv_block(prev_layer, current_size, conv_kernel, net_setup, block_name, n)
        if current_size == net_setup.last_layer_size: break
        current_size = max(net_setup.last_layer_size, int(current_size / net_setup.decay_factor))
        n += 1
    return prev_layer

In [10]:
def create_cell_model(net_config, loc):
    comp_net_setup = NetSetup('PReLU', [1, 2], 0.2, 1024, 64, 1.6, None, False)
            
    input_layers = []
    high_level_features = []
    
    reduced_inputs = []
    for comp_id in range(len(net_config.comp_names)):
        comp_name = net_config.comp_names[comp_id]
        n_comp_features = len(input_cell_external_branches) + len(net_config.comp_branches[comp_id])
        input_layer_comp = Input(name="input_{}_{}".format(loc, comp_name),
                                 shape=(1, 1, n_comp_features))
        input_layers.append(input_layer_comp)
        comp_net_setup.RecalcLayerSizes(n_comp_features, 2, 1)
        reduced_comp = reduce_n_features_2d(input_layer_comp, comp_net_setup, "{}_{}".format(loc, comp_name))
        reduced_inputs.append(reduced_comp)

    cell_output_size = 64
    if len(net_config.comp_names) > 1:
        conv_all_start = Concatenate(name="{}_cell_concat".format(loc), axis=3)(reduced_inputs)
        comp_net_setup.first_layer_size = conv_all_start.shape.as_list()[3]
        comp_net_setup.last_layer_size = 64
        prev_layer = reduce_n_features_2d(conv_all_start, comp_net_setup, "{}_all".format(loc))
    else:
        prev_layer = reduced_inputs[0]
        
    model = Model(input_layers, prev_layer, name="DeepTau2017v2")
    return model

In [11]:
model_name = "DeepTau2017v2p6"
inner_model = create_cell_model(netConf_full, 'inner')
inner_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_inner_egamma (InputLayer) (None, 1, 1, 86)     0                                            
__________________________________________________________________________________________________
input_inner_muon (InputLayer)   (None, 1, 1, 64)     0                                            
__________________________________________________________________________________________________
input_inner_hadrons (InputLayer (None, 1, 1, 38)     0                                            
__________________________________________________________________________________________________
inner_egamma_conv_1 (

In [47]:
inner_model.load_weights('../../../../output/networks/2017v2p6/DeepTau2017v2p6_step1_e6.h5', by_name=True)

In [48]:
inner_model.save('../../../../output/networks/2017v2p6/DeepTau2017v2p6_step1_e6_inner.h5')

In [12]:
model_name = "DeepTau2017v2p6"
outer_model = create_cell_model(netConf_full, 'outer')
outer_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_outer_egamma (InputLayer) (None, 1, 1, 86)     0                                            
__________________________________________________________________________________________________
input_outer_muon (InputLayer)   (None, 1, 1, 64)     0                                            
__________________________________________________________________________________________________
input_outer_hadrons (InputLayer (None, 1, 1, 38)     0                                            
__________________________________________________________________________________________________
outer_egamma_conv_1 (Conv2D)    (None, 1, 1, 207)    18009       input_outer_egamma[0][0]         
__________________________________________________________________________________________________
outer_muon

In [50]:
outer_model.load_weights('../../../../output/networks/2017v2p6/DeepTau2017v2p6_step1_e6.h5', by_name=True)

In [51]:
outer_model.save('../../../../output/networks/2017v2p6/DeepTau2017v2p6_step1_e6_outer.h5')

In [13]:
def create_core_model(net_config):
    tau_net_setup = NetSetup('PReLU', None, 0.2, 128, 128, 1.4, None, False)
    comp_net_setup = NetSetup('PReLU', [1, 2], 0.2, 1024, 64, 1.6, None, False)
    dense_net_setup = NetSetup('PReLU', None, 0.2, 200, 64, 1.4, None, False)
            
    input_layers = []
    high_level_features = []

    if len(net_config.tau_branches) > 0:
        input_layer_tau = Input(name="input_tau", shape=(len(net_config.tau_branches),))
        input_layers.append(input_layer_tau)
        tau_net_setup.RecalcLayerSizes(len(net_config.tau_branches), 2, 1)
        processed_tau = reduce_n_features_1d(input_layer_tau, tau_net_setup, 'tau')
        #processed_tau = dense_block_sequence(input_layer_tau, tau_net_setup, 4, 'tau')
        high_level_features.append(processed_tau)
    
    for loc in net_config.cell_locations:
        cell_output_size = 64
        input_layer_loc = Input(name="input_{}".format(loc),
                                shape=(n_cells_eta[loc], n_cells_phi[loc], cell_output_size))
        input_layers.append(input_layer_loc)
        prev_layer = input_layer_loc
        
        window_size = 3
        current_size = n_cells_eta[loc]
        n = 1
        while current_size > 1:
            win_size = min(current_size, window_size)
            prev_layer = conv_block(prev_layer, cell_output_size, (win_size, win_size), comp_net_setup,
                                    "{}_all_{}x{}".format(loc, win_size, win_size), n)
            n += 1
            current_size -= window_size - 1
            
        cells_flatten = Flatten(name="{}_cells_flatten".format(loc))(prev_layer)
        high_level_features.append(cells_flatten)
        
    if len(high_level_features) > 1:
        features_concat = Concatenate(name="features_concat", axis=1)(high_level_features)
    else:
        features_concat = high_level_features[0]
    if net_config.final:
        final_dense = dense_block_sequence(features_concat, dense_net_setup, 4, 'final')
        output_layer = Dense(n_outputs, name="final_dense_last",
                             kernel_initializer=dense_net_setup.kernel_init)(final_dense)

    else:
        final_dense = dense_block(features_concat, 1024, dense_net_setup,
                                  'tmp_{}'.format(net_config.name), 1)
        output_layer = Dense(n_outputs, name="tmp_{}_dense_last".format(net_config.name),
                             kernel_initializer=dense_net_setup.kernel_init)(final_dense)
    softmax_output = Activation("softmax", name="main_output")(output_layer)
    
    print(input_layers)
    model = Model(input_layers, softmax_output, name="DeepTau2017v2")
    return model

In [14]:
model_name = "DeepTau2017v2p6"
core_model = create_core_model(netConf_full)
core_model.summary()

[<tf.Tensor 'input_tau:0' shape=(?, 47) dtype=float32>, <tf.Tensor 'input_inner:0' shape=(?, 11, 11, 64) dtype=float32>, <tf.Tensor 'input_outer:0' shape=(?, 21, 21, 64) dtype=float32>]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_outer (InputLayer)        (None, 21, 21, 64)   0                                            
__________________________________________________________________________________________________
outer_all_3x3_conv_1 (Conv2D)   (None, 19, 19, 64)   36928       input_outer[0][0]                
__________________________________________________________________________________________________
outer_all_3x3_norm_1 (BatchNorm (None, 19, 19, 64)   256         outer_all_3x3_conv_1[0][0]       
__________________________________________________________________________________________________
outer_all_3x3_activati

In [61]:
core_model.load_weights('../../../../output/networks/2017v2p6/DeepTau2017v2p6_step1_e6.h5', by_name=True)

In [62]:
core_model.save('../../../../output/networks/2017v2p6/DeepTau2017v2p6_step1_e6_core.h5')

In [15]:
def create_conv_model(net_config, loc):
    comp_net_setup = NetSetup('PReLU', [1, 2], 0.2, 1024, 64, 1.6, None, False)
            
    input_layers = []
    high_level_features = []

    
    cell_output_size = 64
    input_layer_loc = Input(name="input_{}".format(loc),
                            shape=(n_cells_eta[loc], n_cells_phi[loc], cell_output_size))
    input_layers.append(input_layer_loc)
    prev_layer = input_layer_loc

    window_size = 3
    current_size = n_cells_eta[loc]
    n = 1
    while current_size > 1:
        win_size = min(current_size, window_size)
        prev_layer = conv_block(prev_layer, cell_output_size, (win_size, win_size), comp_net_setup,
                                "{}_all_{}x{}".format(loc, win_size, win_size), n)
        n += 1
        current_size -= window_size - 1

    cells_flatten = Flatten(name="{}_cells_flatten".format(loc))(prev_layer)
    model = Model(input_layers, cells_flatten, name="DeepTau2017v2")
    return model

In [16]:
model_name = "DeepTau2017v2p6"
inner_conv_model = create_conv_model(netConf_full, 'inner')
inner_conv_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_inner (InputLayer)     (None, 11, 11, 64)        0         
_________________________________________________________________
inner_all_3x3_conv_1 (Conv2D (None, 9, 9, 64)          36928     
_________________________________________________________________
inner_all_3x3_norm_1 (BatchN (None, 9, 9, 64)          256       
_________________________________________________________________
inner_all_3x3_activation_1 ( (None, 9, 9, 64)          64        
_________________________________________________________________
inner_all_3x3_dropout_1 (Dro (None, 9, 9, 64)          0         
_________________________________________________________________
inner_all_3x3_conv_2 (Conv2D (None, 7, 7, 64)          36928     
_________________________________________________________________
inner_all_3x3_norm_2 (BatchN (None, 7, 7, 64)          256       
__________

In [17]:
inner_conv_model.load_weights('../../../../output/networks/2017v2p6/DeepTau2017v2p6_step1_e6.h5', by_name=True)
inner_conv_model.save('../../../../output/networks/2017v2p6/DeepTau2017v2p6_step1_e6_inner_conv.h5')

In [18]:
model_name = "DeepTau2017v2p6"
outer_conv_model = create_conv_model(netConf_full, 'outer')
outer_conv_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_outer (InputLayer)     (None, 21, 21, 64)        0         
_________________________________________________________________
outer_all_3x3_conv_1 (Conv2D (None, 19, 19, 64)        36928     
_________________________________________________________________
outer_all_3x3_norm_1 (BatchN (None, 19, 19, 64)        256       
_________________________________________________________________
outer_all_3x3_activation_1 ( (None, 19, 19, 64)        64        
_________________________________________________________________
outer_all_3x3_dropout_1 (Dro (None, 19, 19, 64)        0         
_________________________________________________________________
outer_all_3x3_conv_2 (Conv2D (None, 17, 17, 64)        36928     
_________________________________________________________________
outer_all_3x3_norm_2 (BatchN (None, 17, 17, 64)        256       
__________

In [19]:
outer_conv_model.load_weights('../../../../output/networks/2017v2p6/DeepTau2017v2p6_step1_e6.h5', by_name=True)
outer_conv_model.save('../../../../output/networks/2017v2p6/DeepTau2017v2p6_step1_e6_outer_conv.h5')

In [11]:
def create_tau_model(net_config):
    tau_net_setup = NetSetup('PReLU', None, 0.2, 128, 128, 1.4, None, False)
    comp_net_setup = NetSetup('PReLU', [1, 2], 0.2, 1024, 64, 1.6, None, False)
    dense_net_setup = NetSetup('PReLU', None, 0.2, 200, 64, 1.4, None, False)
            
    input_layers = []
    high_level_features = []

    if len(net_config.tau_branches) > 0:
        input_layer_tau = Input(name="input_tau", shape=(len(net_config.tau_branches),))
        input_layers.append(input_layer_tau)
        tau_net_setup.RecalcLayerSizes(len(net_config.tau_branches), 2, 1)
        processed_tau = reduce_n_features_1d(input_layer_tau, tau_net_setup, 'tau')
        #processed_tau = dense_block_sequence(input_layer_tau, tau_net_setup, 4, 'tau')
        high_level_features.append(processed_tau)
    
    print(input_layers)
    model = Model(input_layers, processed_tau, name="DeepTau2017v2")
    return model

In [12]:
model_name = "DeepTau2017v2p6"
tau_model = create_tau_model(netConf_full)
tau_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[<tf.Tensor 'input_tau:0' shape=(?, 47) dtype=float32>]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_tau (InputLayer)       (None, 47)                0         
_________________________________________________________________
tau_dense_1 (Dense)          (None, 113)               5424      
_________________________________________________________________
tau_norm_1 (BatchNormalizati (None, 113)               452       
_________________________________________________________________
tau_activation_1 (PReLU)     (None, 113)               113       
_________________________________________________________________
tau_dropout_1 (Dropout)      (None, 113)               0         
_______________________________

In [13]:
def create_dense_model(net_config):
    tau_net_setup = NetSetup('PReLU', None, 0.2, 128, 128, 1.4, None, False)
    comp_net_setup = NetSetup('PReLU', [1, 2], 0.2, 1024, 64, 1.6, None, False)
    dense_net_setup = NetSetup('PReLU', None, 0.2, 200, 64, 1.4, None, False)
            

    features_concat = Input(name="concat_input", shape=(185,))
    final_dense = dense_block_sequence(features_concat, dense_net_setup, 4, 'final')
    output_layer = Dense(n_outputs, name="final_dense_last",
                         kernel_initializer=dense_net_setup.kernel_init)(final_dense)

    softmax_output = Activation("softmax", name="main_output")(output_layer)
    
    model = Model(features_concat, softmax_output, name="DeepTau2017v2")
    return model

In [14]:
model_name = "DeepTau2017v2p6"
dense_model = create_dense_model(netConf_full)
dense_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
concat_input (InputLayer)    (None, 185)               0         
_________________________________________________________________
final_dense_1 (Dense)        (None, 200)               37200     
_________________________________________________________________
final_norm_1 (BatchNormaliza (None, 200)               800       
_________________________________________________________________
final_activation_1 (PReLU)   (None, 200)               200       
_________________________________________________________________
final_dropout_1 (Dropout)    (None, 200)               0         
_________________________________________________________________
final_dense_2 (Dense)        (None, 200)               40200     
_________________________________________________________________
final_norm_2 (BatchNormaliza (None, 200)               800       
__________

In [162]:
orig_graph = load_graph('../../../../output/networks/2017v2p6/DeepTau2017v2p6_step1_e6.pb')
graph = load_graph('../../../../output/networks/2017v2p6/DeepTau2017v2p6_step1_e6_core.pb')

In [146]:
w_tensors = []
orig_w_tensors = []
for op in graph.get_operations():
    if "activation" in op.name or "batchnorm" in op.name or "input" in op.name: continue
    orig_op = orig_graph.get_operation_by_name(op.name)
    op_values = op.values()
    orig_op_values = orig_op.values()
    for n in range(len(op_values)):
        x = op_values[n]
        if len(x.shape) > 0 and x.shape[0].value is not None and "FusedBatchNorm" not in x.name \
           and "flatten" not in x.name:
            w_tensors.append(x)
            orig_w_tensors.append(orig_op_values[n])
w_tensors

[<tf.Tensor 'deepTau/outer_all_3x3_conv_1/kernel:0' shape=(3, 3, 64, 64) dtype=float32>,
 <tf.Tensor 'deepTau/outer_all_3x3_conv_1/bias:0' shape=(64,) dtype=float32>,
 <tf.Tensor 'deepTau/outer_all_3x3_norm_1/gamma:0' shape=(64,) dtype=float32>,
 <tf.Tensor 'deepTau/outer_all_3x3_norm_1/beta:0' shape=(64,) dtype=float32>,
 <tf.Tensor 'deepTau/outer_all_3x3_norm_1/moving_mean:0' shape=(64,) dtype=float32>,
 <tf.Tensor 'deepTau/outer_all_3x3_norm_1/moving_variance:0' shape=(64,) dtype=float32>,
 <tf.Tensor 'deepTau/outer_all_3x3_conv_2/kernel:0' shape=(3, 3, 64, 64) dtype=float32>,
 <tf.Tensor 'deepTau/outer_all_3x3_conv_2/bias:0' shape=(64,) dtype=float32>,
 <tf.Tensor 'deepTau/outer_all_3x3_norm_2/gamma:0' shape=(64,) dtype=float32>,
 <tf.Tensor 'deepTau/outer_all_3x3_norm_2/beta:0' shape=(64,) dtype=float32>,
 <tf.Tensor 'deepTau/outer_all_3x3_norm_2/moving_mean:0' shape=(64,) dtype=float32>,
 <tf.Tensor 'deepTau/outer_all_3x3_norm_2/moving_variance:0' shape=(64,) dtype=float32>,
 <tf

In [143]:
orig_graph.get_operations()

[<tf.Operation 'deepTau/outer_egamma_conv_1/kernel' type=Const>,
 <tf.Operation 'deepTau/outer_egamma_conv_1/bias' type=Const>,
 <tf.Operation 'deepTau/outer_muon_conv_1/kernel' type=Const>,
 <tf.Operation 'deepTau/outer_muon_conv_1/bias' type=Const>,
 <tf.Operation 'deepTau/outer_hadrons_conv_1/kernel' type=Const>,
 <tf.Operation 'deepTau/outer_hadrons_conv_1/bias' type=Const>,
 <tf.Operation 'deepTau/outer_egamma_norm_1/gamma' type=Const>,
 <tf.Operation 'deepTau/outer_egamma_norm_1/beta' type=Const>,
 <tf.Operation 'deepTau/outer_egamma_norm_1/moving_mean' type=Const>,
 <tf.Operation 'deepTau/outer_egamma_norm_1/moving_variance' type=Const>,
 <tf.Operation 'deepTau/outer_muon_norm_1/gamma' type=Const>,
 <tf.Operation 'deepTau/outer_muon_norm_1/beta' type=Const>,
 <tf.Operation 'deepTau/outer_muon_norm_1/moving_mean' type=Const>,
 <tf.Operation 'deepTau/outer_muon_norm_1/moving_variance' type=Const>,
 <tf.Operation 'deepTau/outer_hadrons_norm_1/gamma' type=Const>,
 <tf.Operation 'dee

In [155]:
graph.get_operations()

[<tf.Operation 'deepTau/inner_egamma_conv_1/kernel' type=Const>,
 <tf.Operation 'deepTau/inner_egamma_conv_1/bias' type=Const>,
 <tf.Operation 'deepTau/inner_muon_conv_1/kernel' type=Const>,
 <tf.Operation 'deepTau/inner_muon_conv_1/bias' type=Const>,
 <tf.Operation 'deepTau/inner_hadrons_conv_1/kernel' type=Const>,
 <tf.Operation 'deepTau/inner_hadrons_conv_1/bias' type=Const>,
 <tf.Operation 'deepTau/inner_egamma_norm_1/gamma' type=Const>,
 <tf.Operation 'deepTau/inner_egamma_norm_1/beta' type=Const>,
 <tf.Operation 'deepTau/inner_egamma_norm_1/moving_mean' type=Const>,
 <tf.Operation 'deepTau/inner_egamma_norm_1/moving_variance' type=Const>,
 <tf.Operation 'deepTau/inner_muon_norm_1/gamma' type=Const>,
 <tf.Operation 'deepTau/inner_muon_norm_1/beta' type=Const>,
 <tf.Operation 'deepTau/inner_muon_norm_1/moving_mean' type=Const>,
 <tf.Operation 'deepTau/inner_muon_norm_1/moving_variance' type=Const>,
 <tf.Operation 'deepTau/inner_hadrons_norm_1/gamma' type=Const>,
 <tf.Operation 'dee

In [147]:
with graph.as_default() as gr:
    with tf.Session() as sess:
        weights = sess.run(w_tensors)

In [148]:
with orig_graph.as_default() as gr:
    with tf.Session() as sess:
        orig_weights = sess.run(orig_w_tensors)

In [150]:
for n in range(len(weights)):
    print(w_tensors[n].name, np.amax(np.abs(weights[n] - orig_weights[n])))

deepTau/outer_all_3x3_conv_1/kernel:0 0.0
deepTau/outer_all_3x3_conv_1/bias:0 0.0
deepTau/outer_all_3x3_norm_1/gamma:0 0.0
deepTau/outer_all_3x3_norm_1/beta:0 0.0
deepTau/outer_all_3x3_norm_1/moving_mean:0 0.0
deepTau/outer_all_3x3_norm_1/moving_variance:0 0.0
deepTau/outer_all_3x3_conv_2/kernel:0 0.0
deepTau/outer_all_3x3_conv_2/bias:0 0.0
deepTau/outer_all_3x3_norm_2/gamma:0 0.0
deepTau/outer_all_3x3_norm_2/beta:0 0.0
deepTau/outer_all_3x3_norm_2/moving_mean:0 0.0
deepTau/outer_all_3x3_norm_2/moving_variance:0 0.0
deepTau/outer_all_3x3_conv_3/kernel:0 0.0
deepTau/outer_all_3x3_conv_3/bias:0 0.0
deepTau/outer_all_3x3_norm_3/gamma:0 0.0
deepTau/outer_all_3x3_norm_3/beta:0 0.0
deepTau/outer_all_3x3_norm_3/moving_mean:0 0.0
deepTau/outer_all_3x3_norm_3/moving_variance:0 0.0
deepTau/outer_all_3x3_conv_4/kernel:0 0.0
deepTau/outer_all_3x3_conv_4/bias:0 0.0
deepTau/outer_all_3x3_norm_4/gamma:0 0.0
deepTau/outer_all_3x3_norm_4/beta:0 0.0
deepTau/outer_all_3x3_norm_4/moving_mean:0 0.0
deepTau

In [104]:
test_w = inner_model.get_layer(name='inner_all_activation_1').get_weights()

In [99]:
orig_model = LoadModel('../../../../output/networks/2017v2p6/DeepTau2017v2p6_step1_e6.h5', compile=False)

In [107]:
orig_w = orig_model.get_layer(name='inner_all_activation_1').get_weights()

In [110]:
test_w[0] - orig_w[0]

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [139]:
with graph.as_default() as gr:
    with tf.Session() as sess:
        zero_output = sess.run(graph.get_tensor_by_name('deepTau/inner_all_activation_4/add:0'),
                               feed_dict= {
                                   'deepTau/input_inner_muon:0': np.zeros((1, 1, 1, 64)),
                                   'deepTau/input_inner_hadrons:0': np.zeros((1, 1, 1, 38)),
                                   'deepTau/input_inner_egamma:0': np.zeros((1, 1, 1, 86)),
                               })

In [133]:
with orig_graph.as_default() as gr:
    with tf.Session() as sess:
        orig_zero_output = sess.run(orig_graph.get_tensor_by_name('deepTau/inner_all_activation_4/add:0'),
                               feed_dict= {
                                   'deepTau/input_inner_muon:0': np.zeros((1, 11, 11, 64)),
                                   'deepTau/input_inner_hadrons:0': np.zeros((1, 11, 11, 38)),
                                   'deepTau/input_inner_egamma:0': np.zeros((1, 11, 11, 86)),
                               })

In [134]:
print(zero_output.shape, orig_zero_output.shape)

(1, 1, 1, 64) (1, 11, 11, 64)


In [136]:
zero_output

array([[[[-3.4153205e-01,  2.7983323e-01,  3.3936810e-01,
           2.6720908e-01, -4.0494433e-01,  2.9774749e-01,
           3.8688186e-01,  1.3517329e-01,  7.5788379e-02,
           3.2355881e-01,  2.3890865e-01,  3.7686965e-01,
           1.3836404e+00, -5.0892520e-01,  1.2147540e+00,
          -7.6665545e-01,  1.2904071e+00, -1.7790170e-01,
           4.2508158e-01, -1.1790282e+00,  5.3043944e-01,
           1.1485299e+00,  2.9395887e-01,  3.0837867e-01,
           1.8521161e+00,  9.6835285e-01,  2.8270960e-01,
           1.8168612e+00, -3.9322859e-01, -4.2629471e-01,
           4.6222180e-01, -2.0216801e-03,  1.4596337e-01,
           9.0908563e-01,  7.7557981e-01,  4.2415512e-01,
          -1.7613149e-01, -5.1867223e-01, -1.4167570e-01,
          -1.0154943e+00, -6.3860732e-01,  1.2392056e-01,
          -1.9441524e-01, -2.3127201e-01,  1.1322395e+00,
           2.6782990e-01, -6.8590093e-01,  4.7952279e-02,
          -4.9580187e-01, -1.1954730e+00,  9.9887443e-01,
          -6.4

In [140]:
np.amax(orig_zero_output[0, 2, 4, :] - zero_output)

3.5762787e-07

In [156]:
for i in orig_graph.get_operation_by_name('deepTau/inner_egamma_conv_1/kernel').inputs:
    print(i)

In [157]:
orig_graph.get_tensor_by_name('deepTau/inner_egamma_conv_1/kernel:0')

<tf.Tensor 'deepTau/inner_egamma_conv_1/kernel:0' shape=(1, 1, 86, 207) dtype=float32>

In [158]:
op = orig_graph.get_operation_by_name('deepTau/inner_egamma_conv_1/kernel')

In [165]:
graph.get_operations()

[<tf.Operation 'deepTau/outer_all_3x3_conv_1/kernel' type=Const>,
 <tf.Operation 'deepTau/outer_all_3x3_conv_1/bias' type=Const>,
 <tf.Operation 'deepTau/outer_all_3x3_norm_1/gamma' type=Const>,
 <tf.Operation 'deepTau/outer_all_3x3_norm_1/beta' type=Const>,
 <tf.Operation 'deepTau/outer_all_3x3_norm_1/moving_mean' type=Const>,
 <tf.Operation 'deepTau/outer_all_3x3_norm_1/moving_variance' type=Const>,
 <tf.Operation 'deepTau/outer_all_3x3_conv_2/kernel' type=Const>,
 <tf.Operation 'deepTau/outer_all_3x3_conv_2/bias' type=Const>,
 <tf.Operation 'deepTau/outer_all_3x3_norm_2/gamma' type=Const>,
 <tf.Operation 'deepTau/outer_all_3x3_norm_2/beta' type=Const>,
 <tf.Operation 'deepTau/outer_all_3x3_norm_2/moving_mean' type=Const>,
 <tf.Operation 'deepTau/outer_all_3x3_norm_2/moving_variance' type=Const>,
 <tf.Operation 'deepTau/outer_all_3x3_conv_3/kernel' type=Const>,
 <tf.Operation 'deepTau/outer_all_3x3_conv_3/bias' type=Const>,
 <tf.Operation 'deepTau/outer_all_3x3_norm_3/gamma' type=Con

In [184]:
with graph.as_default() as gr:
    with tf.Session() as sess:
        zero_output = sess.run(graph.get_tensor_by_name('deepTau/tau_dense_1/bias:0'),
                               feed_dict= {
                                   'deepTau/input_inner:0': np.zeros((1, 11, 11, 64)),
                                   'deepTau/input_outer:0': np.zeros((1, 21, 21, 64))
                               })
print(zero_output)

[-2.0570154e-03 -5.0227558e-03 -3.9292602e-03 -9.1202408e-03
  6.6446117e-03 -6.0116760e-02 -4.7941925e-03 -7.7092354e-03
 -2.0869890e-02  1.5228596e-02 -3.9257050e-02 -4.2718370e-02
  9.8306974e-03  2.3221709e-02 -4.5068424e-02  8.4627643e-02
 -2.4317075e-02  1.7266396e-03 -1.9816547e-03 -1.6043956e-03
 -9.3630105e-03  2.9751955e-02  4.7717844e-03  3.6520825e-03
  2.3412924e-02 -2.7577430e-02 -1.4300602e-02  2.5707729e-02
 -5.4116622e-03 -1.0513316e-03  1.3275772e-02  5.8405800e-03
 -2.4988055e-02  1.6775634e-02 -3.4314925e-03  5.5084173e-03
  1.1601421e-02  2.8195879e-03  1.7089028e-02  2.3744423e-02
  5.7679659e-04 -6.0707564e-03  9.5828949e-03  2.1789758e-02
  4.1340679e-02 -8.1450269e-03  7.1673808e-03  1.0108606e-02
  2.1742254e-03 -1.5618774e-02  1.1865381e-02 -2.1857809e-04
  8.8280560e-03 -2.7078398e-02 -1.2511135e-02  2.3999728e-02
 -2.1157900e-02  8.7953629e-03 -8.1386166e-03  1.3368088e-02
 -4.0510971e-02  1.1861235e-02  2.4367525e-04 -1.9663533e-02
  1.4308085e-02  4.60573

In [176]:
for n in graph.as_graph_def().node:
    print(n.name)

deepTau/outer_all_3x3_conv_1/kernel
deepTau/outer_all_3x3_conv_1/bias
deepTau/outer_all_3x3_norm_1/gamma
deepTau/outer_all_3x3_norm_1/beta
deepTau/outer_all_3x3_norm_1/moving_mean
deepTau/outer_all_3x3_norm_1/moving_variance
deepTau/outer_all_3x3_conv_2/kernel
deepTau/outer_all_3x3_conv_2/bias
deepTau/outer_all_3x3_norm_2/gamma
deepTau/outer_all_3x3_norm_2/beta
deepTau/outer_all_3x3_norm_2/moving_mean
deepTau/outer_all_3x3_norm_2/moving_variance
deepTau/outer_all_3x3_conv_3/kernel
deepTau/outer_all_3x3_conv_3/bias
deepTau/outer_all_3x3_norm_3/gamma
deepTau/outer_all_3x3_norm_3/beta
deepTau/outer_all_3x3_norm_3/moving_mean
deepTau/outer_all_3x3_norm_3/moving_variance
deepTau/outer_all_3x3_conv_4/kernel
deepTau/outer_all_3x3_conv_4/bias
deepTau/outer_all_3x3_norm_4/gamma
deepTau/outer_all_3x3_norm_4/beta
deepTau/outer_all_3x3_norm_4/moving_mean
deepTau/outer_all_3x3_norm_4/moving_variance
deepTau/outer_all_3x3_conv_5/kernel
deepTau/outer_all_3x3_conv_5/bias
deepTau/outer_all_3x3_norm_5/g

In [182]:
dir(graph.as_graph_def().node)

['MergeFrom',
 '__class__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'add',
 'extend',
 'pop',
 'remove',
 'sort']